### Geenrative Art: Texture generation and mapping with Blender API
<br> by Hamed Shah-Hosseini, Twitter:@chekaad, Instagram:@ostad.ai
<br>https://github.com/ostad-ai/Generative-Art

In [2]:
from IPython.display import Image

In [4]:
# uncomment and run the following code in the Text Editor of Blender
import bpy
import numpy as np
#delete previous data in Blender
def delete_all():
    for datas in (bpy.data.objects,bpy.data.meshes,
                bpy.data.lights, bpy.data.cameras,
                bpy.data.images, bpy.data.materials,
                bpy.data.actions, bpy.data.curves):
            for data in datas:
                datas.remove(data)
delete_all()
bpy.context.scene.render.film_transparent = True

# add camer and light
bpy.ops.object.camera_add(location=(0,-16,10),rotation=(1.,0,0))
bpy.ops.object.light_add(type='POINT', radius=1, align='WORLD', location=(3,-.4,4.5))
bpy.context.object.data.energy =1000

#--add text
bpy.ops.object.text_add(location=(-5,-3,.3), rotation=(1, 0,0))
bpy.context.object.data.body="Texture Generation and mapping with Blender API"
bpy.context.object.data.extrude=.1
bpy.context.object.data.size=.5

# image (texture) size: width and height
w, h = 600,600
pixel_data = np.ones((h,w, 4), 'f')

#process pixels
center=np.array([h/2.,w/2.]) #center of image
for x in range(w):
    for y in range(h):
        dist=np.array([y,x])-center
        dist=np.sqrt(np.sum(dist**2))
        dist%=50
        factor=np.exp(-dist/120)
        pixel_data[y,x ,:3]=[factor**3,factor**2,factor]
#end of processing pixels

output_image_name = "My OutImage"
# Create output image
output_image = bpy.data.images.new(output_image_name, alpha=True,width=w, height=h)
# Copy of pixel data from numpy array back to the output image
output_image.pixels.foreach_set(pixel_data.ravel())
output_image.update()

#create plane and cube
bpy.ops.mesh.primitive_plane_add(location=(0,0,0))
bpy.ops.transform.resize(value=(10,10,1))
mat=bpy.data.materials.new(name="material_plane")
mat.diffuse_color=(210/255,105/255,30/255,1) #chocolate color
bpy.context.object.data.materials.append(mat)
bpy.ops.mesh.primitive_cube_add(location=(0,0,1))

# texture mapping
bpy.ops.object.editmode_toggle()
bpy.ops.uv.cube_project(cube_size=1)
bpy.ops.object.editmode_toggle()

# material settings
mat=bpy.data.materials.new(name="material_cube")
mat.use_nodes=True
bsdf = mat.node_tree.nodes["Principled BSDF"]
texImage = mat.node_tree.nodes.new('ShaderNodeTexImage')
texImage.image = output_image
mat.node_tree.links.new(bsdf.inputs['Base Color'], texImage.outputs['Color'])
bpy.context.object.data.materials.append(mat)

# to animate
key_frames=[1,50,100,150,200,250]
delta_angle=2*np.pi/(len(key_frames)-1)
for fr in key_frames:
    bpy.ops.transform.rotate(value=delta_angle, orient_axis='Z')
    bpy.context.object.keyframe_insert(data_path='rotation_euler',frame=fr)

In [3]:
Image('./imagesd/image-generation-snapshot.png',width=400,height=200)